In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import json
import random
import tensorflow as tf

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101V2,ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, LSTM, Input, Bidirectional, Flatten, Dense, Dropout, BatchNormalization,GlobalAveragePooling2D,TimeDistributed

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
nframe=16
curr_path = "C:/Users/VISHWANATHAN VIVEK S/Desktop/Projects/VideoCaptioning"
test_path = curr_path + "/Emotion_Detector.mp4"
vid=cv2.VideoCapture(test_path)
frame_cnt = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count=0

print(os.path.exists(curr_path + "/frames"))

try:
    if not os.path.exists(curr_path + "/frames"):
        os.makedirs(curr_path + "/frames")
except OSError:
    print ('Error: Creating directory of data')
        
while(True):
    ret, frame = vid.read()
    if ret :
        if (frame_count<frame_cnt and frame_count%(frame_cnt//nframe)==0):
            name = curr_path + '/frames' + '/' + str(frame_count//(frame_cnt//nframe)) + '.jpg'
            print(name)
            cv2.imwrite(name, frame)
            frame_count+=1
        else :
            frame_count+=1
            print(frame_count)
            continue
    else :
        break
    
vid.release()
cv2.destroyAllWindows()

False


In [5]:
def enc_model(inputs,h_image,w_image):

  base_model=ResNet50V2(include_top=False, input_shape=(h_image, w_image, 3),weights='imagenet')
  model1=Sequential()
  model1.add(TimeDistributed(base_model))
  model1.add(TimeDistributed(GlobalAveragePooling2D()))
  model1.add(TimeDistributed(Dropout(0.5)))
  model1.add(TimeDistributed(Dense(1024,activation='relu')))
  model1.add(Bidirectional(LSTM(256, return_sequences=True)))
  model1.add(Dropout(0.5))
  model1.add(Bidirectional(LSTM(64, return_sequences=True)))

  for layers in base_model.layers[:-2]:
    layers.trainable=False

  enc_output=model1(inputs)
  return enc_output #shape is batch_size, nframes, hidden_state


In [6]:
h_image=720
w_image=1280
nframe=16

In [7]:
print(enc_model(tf.random.normal([1,16,h_image,w_image,3]),h_image,w_image).shape)

94683136/94668760 [==============================] - 1s 0us/step
(1, 16, 128)


In [8]:
class pre_enc_model(tf.keras.layers.Layer):
  def __init__(self):
    super(pre_enc_model, self).__init__()
    self.time_distributed1=TimeDistributed(Dense(1024,activation='relu'))
    self.lstm1=Bidirectional(LSTM(256, return_sequences=True))
    self.dropout=Dropout(0.5)
    self.lstm2=Bidirectional(LSTM(64, return_sequences=True))

  def call(self, inputs):
    x=self.time_distributed1(inputs)
    x=self.lstm1(x)
    x=self.dropout(x)
    enc_output=self.lstm2(x)
    enc_hidden=enc_output[:,-1,:]

    return enc_output, enc_hidden


In [9]:
model4=pre_enc_model()
x,y=model4(tf.random.normal([2,80,4096]))
print(x.shape,y.shape)

(2, 80, 128) (2, 128)


In [10]:
# def pre_enc_model(inputs):
#   model2=Sequential()
#   model2.add(TimeDistributed(Dense(1024,activation='relu')))
#   model2.add(Bidirectional(LSTM(256, return_sequences=True)))
#   model2.add(Dropout(0.5))
#   model2.add(Bidirectional(LSTM(64, return_sequences=True)))

#   enc_output1=model2(inputs)
#   return enc_output1

In [11]:

class Attention(tf.keras.layers.Layer):
  def __init__(self, units=128):
    super(Attention, self).__init__()
    self.W1=Dense(units) 
    self.W2=Dense(units)
    self.v=Dense(1)
  def call(self, enc_output, pre_time_step_input):
    # enc_output will be of shape (batch_size, max_seq_length, encoder_output_shape) encoder_output_shape is the length of output vector of each conv layer(here its 512)
    # pre_time_step_input is of shape (batch_size, decoder_output_shape) decoder_output_shape!=encoder_output_shape and it's the the activation which we pass to next layer of LSTM
    pre_time_step_input=tf.expand_dims(pre_time_step_input, axis=1) # To broadcast pre_time_step_input on each input timestep context vector 
    pre_softmax=self.v(tf.nn.tanh(self.W1(enc_output)+self.W2(pre_time_step_input))) #output shape is(batch_size, max_seq_length, 1 )
    pre_softmax=tf.squeeze(pre_softmax, axis=2)
    softmax_output=tf.nn.softmax(pre_softmax, axis=0)

    context_vectors=tf.expand_dims(softmax_output, axis=2)*enc_output
    context_vector=tf.reduce_sum(context_vectors, axis=1)

    return context_vector #shape (batch_size, encoded_vector_output_shape (here its 128))
    



In [12]:
x=tf.random.normal([32,16,128])
y=tf.random.normal([32,128])
attention=Attention()
context_vector=attention(x,y)
print(context_vector.shape)

(32, 128)


In [13]:
# class Decoder(tf.keras.layers.Layer):
#   def __init__(self, attention, p=0.5, embedding_size=300, vocab_size=10000, hidden_size=128, num_layers=1):
#     super(Decoder, self).__init__()
#     self.hidden_size=hidden_size
#     self.num_layers=num_layers # number of layers of LSTM stacked over each other
#     self.attention=attention #attention will be instantiated later on
#     self.embedding=tf.keras.layers.Embedding(vocab_size, embedding_size)
#     self.lstm=LSTM(hidden_size, return_sequences=True, dropout=0.1, return_state=True)
#     self.dense=Dense(vocab_size, activation='softmax')
#     self.dropout=Dropout(p)

#   def call(self, x, encoder_states, pre_time_step_input, cell_state):

#     x=tf.expand_dims(x,1)
#     embedding_vec = self.dropout(self.embedding(x)) #shape is batch_size,1,embedding_size
#     context_vector = self.attention(encoder_states, pre_time_step_input) 
#     context_vector=tf.expand_dims(context_vector, 1) #shape is batch_size, 1, hidden_size
#     context_vec=tf.concat([embedding_vec, context_vector], axis=-1)
#     _, pre_time_step_input , cell_state=self.lstm(context_vec, initial_state=[pre_time_step_input, cell_state])
#     out=tf.concat([pre_time_step_input, tf.squeeze(context_vec, axis=1)], axis=-1)
#     output=self.dense(out)

#     return output, pre_time_step_input, cell_state




In [14]:
# class Decoder(tf.keras.layers.Layer):
#   def __init__(self, attention, p=0.5, embedding_size=300, vocab_size=10000, hidden_size=128, num_layers=1):
#     super(Decoder, self).__init__()
#     self.hidden_size=hidden_size
#     self.num_layers=num_layers # number of layers of LSTM stacked over each other
#     self.attention=attention #attention will be instantiated later on
#     self.embedding=tf.keras.layers.Embedding(vocab_size, embedding_size)
#     self.lstm=LSTM(hidden_size, return_sequences=True, dropout=0.1, return_state=True)
#     self.dense=Dense(vocab_size, activation='softmax')
#     self.dropout=Dropout(p)

#   def call(self, x, encoder_states, pre_time_step_input, cell_state):

#     x=tf.expand_dims(x,1)
#     embedding_vec = self.dropout(self.embedding(x)) #shape is batch_size,1,embedding_size
#     context_vector = self.attention(encoder_states, pre_time_step_input) 
#     context_vector=tf.expand_dims(context_vector, 1) #shape is batch_size, 1, hidden_size
#     context_vec=tf.concat([embedding_vec, context_vector], axis=-1)
#     _, pre_time_step_input , cell_state=self.lstm(context_vec, initial_state=[pre_time_step_input, cell_state])
#     out=tf.concat([pre_time_step_input, tf.squeeze(context_vec, axis=1)], axis=-1)
#     output=self.dense(out)

#     return output, pre_time_step_input, cell_state




In [15]:
# attention=Attention()
# decoder=Decoder(attention, 0.5, 300, 10000, 128, 1)

# x=tf.random.uniform(shape=([32]), minval=1, maxval=32, dtype=tf.int32)
# encoder_states=tf.random.normal([32,16,128])
# pre_time_step_input=tf.random.normal([32,128])
# cell_state=tf.zeros([32,128])

# a,b,c=decoder(x, encoder_states, pre_time_step_input, cell_state)
# print(a.shape,b.shape,c.shape)


In [16]:
class Decoder(tf.keras.Model):

    def __init__(self, vocab_size=10000, embedding_dim=300, dec_units=128, batch_sz=32, num_layers=1, drop_prob=0.1):
        super(Decoder,self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.num_layers = num_layers
        self.drop_prob = drop_prob
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # multiple number of layers
        self.gru_layers = []
        for i in range(self.num_layers):
            self.gru_layers.append(tf.keras.layers.GRU(self.dec_units, return_state = True, return_sequences=True,
                                    recurrent_initializer='glorot_uniform',
                                    dropout = (0 if self.num_layers==1 else self.drop_prob)))
        self.fc = Dense(vocab_size)

        # attention
        self.attention = Attention(self.dec_units)

    def call(self, x, hidden, enc_output):

        # enc output shape = (batch_sz, maxlen, hiddensize)
        context_vector = self.attention(enc_output, hidden)

        # x shape after embedding ( batchsize, 1, embedding_dim)
        # x=tf.argmax(x, axis=-1)
        x = self.embedding(x)
        # x shape after concatenation = (batchsize, 1, embedding_dim + hiddensize)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # passing the concatenated vector to gru
        output, state = self.gru_layers[0](x, initial_state = hidden)
        for i in range(1, self.num_layers-1):
            output, state = self.gru_layers[i](output)
        # output shape = (batchsize*1, hiddensize)
        output = tf.reshape(output, (-1, output.shape[2]))
        # output shape = (batchsize, vocab)
        x = self.fc(output)

        return x, state


# class Decoder(tf.keras.layers.Layer):
#   def __init__(self, attention, p=0.5, embedding_size=300, vocab_size=10000, hidden_size=128, num_layers=1):
#     super(Decoder, self).__init__()
#     self.hidden_size=hidden_size
#     self.num_layers=num_layers # number of layers of LSTM stacked over each other
#     self.attention=attention #attention will be instantiated later on
#     self.embedding=tf.keras.layers.Embedding(vocab_size, embedding_size)
#     self.lstm=LSTM(hidden_size, return_sequences=True, dropout=0.1, return_state=True)
#     self.dense=Dense(vocab_size, activation='softmax')
#     self.dropout=Dropout(p)

#   def call(self, x, encoder_states, pre_time_step_input, cell_state):

#     x=tf.expand_dims(x,1)
#     embedding_vec = self.dropout(self.embedding(x)) #shape is batch_size,1,embedding_size
#     context_vector = self.attention(encoder_states, pre_time_step_input) 
#     context_vector=tf.expand_dims(context_vector, 1) #shape is batch_size, 1, hidden_size
#     context_vec=tf.concat([embedding_vec, context_vector], axis=-1)
#     _, pre_time_step_input , cell_state=self.lstm(context_vec, initial_state=[pre_time_step_input, cell_state])
#     out=tf.concat([pre_time_step_input, tf.squeeze(context_vec, axis=1)], axis=-1)
#     output=self.dense(out)

    # return output, pre_time_step_input, cell_state





In [17]:
encoder=pre_enc_model()
decoder = Decoder()
sample_decoder_output, _ = decoder(tf.random.uniform((32, 1)),
                                      tf.zeros([32,128]), tf.random.normal([32,80,128]))
print(sample_decoder_output.shape)

(32, 10000)


In [18]:
train_path='/content/drive/MyDrive/training_data'
TRAIN_LABEL_PATH = '/content/drive/MyDrive/training_label.json'

# loading the json file for training
with open(TRAIN_LABEL_PATH) as data_file:    
    y_data = json.load(data_file)
# train_list contains all the captions with their video ID
# vocab_list contains all the vocabulary from training data
train_list = []
vocab_list = []
for y in y_data:
  for caption in y['caption']:
    caption = "<bos> " + caption + " <eos>"
    # we are only using sentences whose length lie between 5 and 12
    if len(caption.split())>=12 or len(caption.split())<5:
      continue
    else:
      train_list.append([caption, y['id']])
print(len(train_list))
random.shuffle(train_list)
training_list = train_list
for train in training_list:
    vocab_list.append(train[0])
# Tokenizing the words
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(vocab_list)
print(len(tokenizer.word_index))
x_data = {}
TRAIN_FEATURE_DIR = os.path.join(train_path, 'feat')
# Loading all the numpy arrays at once and saving them in a dictionary
for filename in os.listdir(TRAIN_FEATURE_DIR):
    f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
    x_data[filename[:-4]] = f # -4 to remove .npy from the file names
print(len(training_list))
len(x_data)

19714
4649
19714


1450

In [19]:
print(np.shape(train_list))

(19714, 2)


In [20]:
train_list=train_list[:100]

In [28]:
def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=10000,training_list=train_list, batch_size=32, maxlen=12):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []
    videoId = []
    videoSeq = []
    # separating the videoId and the video captions
    for idx, cap in enumerate(training_list):
        caption = cap[0]
        videoId.append(cap[1])
        videoSeq.append(caption)
    # converting the captions to tokens and padding them to equal sizes
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = np.array(train_sequences)
    train_sequences = pad_sequences(train_sequences, padding='post',truncating='post', maxlen=maxlen)
    max_seq_length = train_sequences.shape[1]
    filesize = len(train_sequences)
    X_data = []
    y_data = []
    vCount = 0
    n = 0
    for i in range(epochs):
      for idx in  range(0,filesize):
        n += 1
        encoder_input_data.append(x_data[videoId[idx]])
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        # decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
        if n == batch_size:
          encoder_input = np.array(encoder_input_data)
          # decoder_input = np.array(decoder_input_data)
          decoder_target = np.array(decoder_target_data)
          encoder_input_data = []
          # decoder_input_data = []
          decoder_target_data = []
          n = 0
          # yield ([encoder_input, decoder_input], decoder_target)
          yield (encoder_input, decoder_target)


In [22]:
# def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=10000,training_list=train_list, batch_size=32, maxlen=12):
#     encoder_input_data = []
#     decoder_input_data = []
#     decoder_target_data = []
#     videoId = []
#     videoSeq = []
#     # separating the videoId and the video captions
#     for idx, cap in enumerate(training_list):
#         caption = cap[0]
#         videoId.append(cap[1])
#         videoSeq.append(caption)
#     # converting the captions to tokens and padding them to equal sizes
#     train_sequences = tokenizer.texts_to_sequences(videoSeq)
#     train_sequences = np.array(train_sequences)
#     train_sequences = pad_sequences(train_sequences, padding='post',truncating='post', maxlen=maxlen)
#     max_seq_length = train_sequences.shape[1]
#     filesize = len(train_sequences)
#     X_data = []
#     y_data = []
#     vCount = 0
#     n = 0
#     for idx in  range(0,filesize):
#       n += 1
#       encoder_input_data.append(x_data[videoId[idx]])
#       y = to_categorical(train_sequences[idx], num_decoder_tokens)
#       # decoder_input_data.append(y[:-1])
#       decoder_target_data.append(y[1:])
#       if n == batch_size:
#         encoder_input = np.array(encoder_input_data)
#         # decoder_input = np.array(decoder_input_data)
#         decoder_target = np.array(decoder_target_data)
#         encoder_input_data = []
#         # decoder_input_data = []
#         decoder_target_data = []
#         n = 0
#         # yield ([encoder_input, decoder_input], decoder_target)
#         yield (encoder_input, decoder_target)


In [29]:
train= load_datatest(train_path=train_path ,batch_size=32, training_list=train_list, x_data=x_data, epochs=150)

In [30]:
next(train)[0].shape, next(train)[1].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


((32, 80, 4096), (32, 11, 10000))

In [31]:
learning_rate=0.001
decoder_learning_rate=1
clip=2

enc_optimizer = tf.keras.optimizers.Adam(lr=learning_rate, 
                                         clipnorm = clip)
dec_optimizer = tf.keras.optimizers.Adam(lr=learning_rate*decoder_learning_rate,
                                         clipnorm = clip)

loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits = False)

def loss_fn(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real,0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [32]:
teach_force=0.5
teacher_forcing_ratio = teach_force
@tf.function
def train_step(inp, rep):
    loss = 0

    with tf.GradientTape() as enc_tape, tf.GradientTape() as dec_tape:
        enc_output, enc_hidden = encoder(inp)
        dec_hidden = enc_hidden

        # dec_input = tf.expand_dims([vocab['SOS']]*BATCH_SIZE, 1)
        dec_input=tf.random.uniform((32, 1))
        
        
        # using teacher_forcing_ratio as a parameter 
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if use_teacher_forcing:
            # teacher forcing - feeding the reply as the next input
            for t in range(1, rep.shape[1]):
                preds, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                loss += loss_fn(rep[:,t,:], preds)
                # teacher forcing
                dec_input = tf.expand_dims(tf.argmax(rep[:, t,:], -1),1)
                

        else:
            # without teacher forcing using the decoder output as next input
            for t in range(1, rep.shape[1]):
                preds, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
                loss += loss_fn(rep[:, t,:], preds)

                # passing the output
                dec_input = tf.expand_dims(tf.math.argmax(preds, axis=1), 1)
                

                

    batch_loss = (loss/ int(rep.shape[1]))
    
    enc_variables = encoder.trainable_variables
    dec_variables = decoder.trainable_variables

    enc_grads = enc_tape.gradient(loss, enc_variables)
    dec_grads = dec_tape.gradient(loss, dec_variables)

    enc_optimizer.apply_gradients(zip(enc_grads, enc_variables))
    dec_optimizer.apply_gradients(zip(dec_grads, dec_variables))


    return batch_loss

In [33]:
print(train_step(next(train)[0], next(train)[1]))
# print(train_step(train.next()[0], train.next()[1]))

tf.Tensor(0.0009131866, shape=(), dtype=float32)


In [34]:
import time
ep = 350
losses = []
val_losses = []
num_batches = np.shape(train_list)[0] // 32
for epoch in range(ep):
    print(f"\nSTARTING EPOCH: {epoch+1}")
    start = time.time()
    enc_hidden = tf.zeros([32, 128])
    total_loss = 0
    val_loss_batch = []

    for (batch,(inp, rep)) in enumerate(train):
        batch_loss = train_step(inp, rep)
        total_loss += batch_loss
        losses.append(batch_loss)

        # if (batch+1)%print_every == 0:
        #     print("\t Batch {}/{} ===> Train Loss {:.4f}".format(batch+1,num_batches,batch_loss.numpy()))
    
    # dataset = dataset.shuffle(BUFFER_SIZE)
    print("Epoch {} completed with Training Loss {:.4f} in {:.4f} sec".format(epoch+1, 
                                                                      total_loss,
                                                                     time.time() - start)) 
# total_loss/steps_per_epoch,                   


STARTING EPOCH: 1


KeyboardInterrupt: ignored

In [ ]:
# import time
# ep = 3
# losses = []
# val_losses = []
# num_batches = np.shape(train_list)[0] // 32
# for epoch in range(ep):
#     print(f"\nSTARTING EPOCH: {epoch+1}")
#     start = time.time()
#     enc_hidden = tf.zeros([32, 128])
#     total_loss = 0
#     val_loss_batch = []

#     for (inp, rep) in train:
#         batch_loss = train_step(inp, rep)
#         total_loss += batch_loss
#         losses.append(batch_loss)

#         # if (batch+1)%print_every == 0:
#         #     print("\t Batch {}/{} ===> Train Loss {:.4f}".format(batch+1,num_batches,batch_loss.numpy()))
    
#     # dataset = dataset.shuffle(BUFFER_SIZE)
#     print("Epoch {} completed with Training Loss {:.4f} in {:.4f} sec".format(epoch+1, 
#                                                                       total_loss,
#                                                                      time.time() - start)) 
# # total_loss/steps_per_epoch,                   

In [ ]:
# outputs=tf.Variable(tf.zeros([32, 12, 10000]))
# # outputs=tf.zeros([32, 12, 10000])
# class Seq2Seq(tf.keras.layers.Layer):
#   def __init__(self, decoder, encoder):
#         super(Seq2Seq, self).__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#   def call(self, source, target, hidden_size=128, teacher_force_ratio=0.5, vocab_size=10000, batch_size=32):

#     target_len=target.shape[1]
#     # outputs = tf.zeros([batch_size, target_len, vocab_size])
#     # outputs=tf.Variable(tf.zeros([batch_size, target_len, vocab_size]))
#     encoder_states=self.encoder(source)
#     # encoder_states=pre_enc_model(source)
#     x = tf.argmax(target[:,0,:] , axis=-1)
#     pre_time_step_input=tf.zeros([batch_size, hidden_size])
#     cell_state=tf.zeros([batch_size, hidden_size])
#     for t in range(1, target_len):
#       output, pre_time_step_input, cell_state = self.decoder(x, encoder_states, pre_time_step_input, cell_state)
#       # outputs[:,t,:] = output
#       outputs[:,t,:].assign(output)
#       best_guess = tf.argmax(output , axis=-1)
#       x = tf.argmax(target[:,t,:] , axis=-1) if random.random() < teacher_force_ratio else best_guess
#     return outputs

In [ ]:
# outputs=tf.Variable(tf.zeros([32, 12, 10000]))
# # outputs=tf.zeros([32, 12, 10000])
# class Seq2Seq(tf.keras.layers.Layer):
  
#   def call(self, source, target, hidden_size=128, teacher_force_ratio=0.5, vocab_size=10000, batch_size=32):

#     target_len=target.shape[1]
#     # outputs = tf.zeros([batch_size, target_len, vocab_size])
#     # outputs=tf.Variable(tf.zeros([batch_size, target_len, vocab_size]))
#     encoder_states=pre_enc_model()(source)
#     # encoder_states=pre_enc_model(source)
#     x = tf.argmax(target[:,0,:] , axis=-1)
#     pre_time_step_input=tf.zeros([batch_size, hidden_size])
#     cell_state=tf.zeros([batch_size, hidden_size])
#     for t in range(1, target_len):
#       output, pre_time_step_input, cell_state = Decoder()(x, encoder_states, pre_time_step_input, cell_state)
#       # outputs[:,t,:] = output
#       outputs[:,t,:].assign(output)
#       best_guess = tf.argmax(output , axis=-1)
#       x = tf.argmax(target[:,t,:] , axis=-1) if random.random() < teacher_force_ratio else best_guess
#     return outputs

In [ ]:
# vocab_size=10000
# embedding_size = 300
# hidden_size = 128
# num_layers = 2
# dec_dropout = 0.2

# time_steps_encoder=80
# num_encoder_tokens=4096
# time_steps_decoder=12

# encoder_net = pre_enc_model()
# attention = Attention()
# decoder_net = Decoder(attention, dec_dropout ,embedding_size, vocab_size, hidden_size, num_layers)


In [ ]:
# vocab_size=10000
# embedding_size = 300
# hidden_size = 128
# num_layers = 2
# dec_dropout = 0.2

# time_steps_encoder=80
# num_encoder_tokens=4096
# time_steps_decoder=12

# encoder_net = pre_enc_model()
# attention = Attention()
# decoder_net = Decoder(dec_dropout ,embedding_size, vocab_size, hidden_size, num_layers)


In [ ]:
# encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
# target= Input(shape=(time_steps_decoder, vocab_size), name="decoder_inputs")
# seq2seq = Seq2Seq(decoder_net , encoder_net)
# outputs=seq2seq(encoder_inputs, target)
# print(outputs.shape)
# # model=Model(encoder_inputs, outputs)

In [ ]:
# tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)

In [28]:
# class Model1(tf.keras.Model):
#   def __init__(self, decoder_net, encoder_net):
#     super(Model1, self).__init__()
#     self.seq2seq=Seq2Seq(decoder_net, encoder_net)

#   def call(self, inputs, target):
#     x=self.seq2seq(inputs, target)
#     return outputs


In [29]:
# model5=Model1(decoder_net, encoder_net)

In [30]:
# output=model5(encoder_inputs, target)

In [31]:
# outputs=tf.Variable(tf.zeros([32, 12, 10000]))
# # outputs=tf.zeros([32, 12, 10000])
# class Seq2Seq(tf.keras.layers.Layer):
#   def __init__(self, decoder, encoder):
#         super(Seq2Seq, self).__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#   def call(self, source, target_len=12, hidden_size=128, teacher_force_ratio=0.5, vocab_size=10000, batch_size=32):

#     target_len=target_len
#     # outputs = tf.zeros([batch_size, target_len, vocab_size])
#     # outputs=tf.Variable(tf.zeros([batch_size, target_len, vocab_size]))
#     encoder_states=self.encoder(source)
#     # encoder_states=pre_enc_model(source)
#     # x = tf.argmax(target[:,0,:] , axis=-1)
#     x=tf.squeeze(tf.argmax(tf.zeros([batch_size,1,vocab_size],dtype=tf.dtypes.float32), axis=-1), axis=-1)
#     pre_time_step_input=tf.zeros([batch_size, hidden_size])
#     cell_state=tf.zeros([batch_size, hidden_size])
#     for t in range(1, target_len):
#       output, pre_time_step_input, cell_state = self.decoder(x, encoder_states, pre_time_step_input, cell_state)
#       # outputs[:,t,:] = output
#       outputs[:,t,:].assign(output)
#       best_guess = tf.argmax(output , axis=-1)
#       # x = tf.argmax(target[:,t,:] , axis=-1) if random.random() < teacher_force_ratio else best_guess
#       x=best_guess
#     return outputs

In [35]:
# encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
# target= Input(shape=(time_steps_decoder, vocab_size), name="decoder_inputs")
# seq2seq = Seq2Seq(decoder_net , encoder_net)
# outputs=seq2seq(encoder_inputs)
# print(outputs.shape)
# # model=Model(encoder_inputs, outputs)

In [36]:
# model=Model(encoder_inputs, outputs)

In [37]:
# model.summary()

In [38]:
# tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True)

In [42]:
# my_callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience = 5, verbose=1, mode='min'),
#               tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, monitor='loss', patience = 5, verbose=1, mode='min' )]
# opt = tf.keras.optimizers.Adam()

In [43]:
# model.compile(metrics=['accuracy'], optimizer=opt, loss='categorical_crossentropy')
# try:
#     model.fit(train, epochs=350, steps_per_epoch=(len(training_list)//32), callbacks=my_callbacks)
# except KeyboardInterrupt:
#     print("\nW: interrupt received, stopping")